In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta
import os
import matplotlib.pyplot as plt

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [3]:
pd.set_option('display.max_rows',500)
pd.set_option('display.max_columns', None)

# Which Months

In [4]:
Monthsinc= '04_20-07_22'

# Amazon Subs (Importing, Merging, Transforming)

In [5]:
SFilepath='C:/Users/sortiz/Onedrive - Cinedigm/Databases/SVOD/Amazon SVOD DB/Amazon Subs/'

Channels = ['Screambox', 'Docurama', 'CONtv', 'Dove Channel']
CountryF = ['US','UK']

In [6]:
#importing Subs for all subs except Fandor

lists = {'2020-04':'2020','2020-05':'2020','2020-06':'2020','2020-07':'2020','2020-08':'2020','2020-09':'2020','2020-10':'2020','2020-11':'2020','2020-12':'2020',
         '2021-01':'2021','2021-02':'2021','2021-03':'2021','2021-04':'2021','2021-05':'2021','2021-06':'2021','2021-07':'2021','2021-08':'2021','2021-09':'2021','2021-10':'2021','2021-11':'2021','2021-12':'2021',
         '2022-01':'2022','2022-02':'2022','2022-03':'2022','2022-04':'2022','2022-05':'2022','2022-06':'2022','2022-07':'2022'}
l=[]

for i in Channels: 
    for key,value in lists.items():
        l = l + [SFilepath + i +'/' + value + '/subs_m_US_' + i + '_' + key + '.csv']
        

In [7]:
#importing Subs for Fandor (Since UK data is always late)

listsF = {'2020-04':'2020','2020-05':'2020','2020-06':'2020','2020-07':'2020','2020-08':'2020','2020-09':'2020','2020-10':'2020','2020-11':'2020','2020-12':'2020',
         '2021-01':'2021','2021-02':'2021','2021-03':'2021','2021-04':'2021','2021-05':'2021','2021-06':'2021','2021-07':'2021','2021-08':'2021','2021-09':'2021','2021-10':'2021','2021-11':'2021','2021-12':'2021',
         '2022-01':'2022','2022-02':'2022','2022-03':'2022','2022-04':'2022','2022-05':'2022','2022-06':'2022','2022-07':'2022'}
lF=[]

for i in CountryF: 
    for key,value in listsF.items():
        lF = lF + [SFilepath + 'Fandor/' + i +'/' + value + '/subs_m_' + i + '_' + 'Fandor' + '_' + key + '.csv']
        

In [8]:
l

['C:/Users/sortiz/Onedrive - Cinedigm/Databases/SVOD/Amazon SVOD DB/Amazon Subs/Screambox/2020/subs_m_US_Screambox_2020-04.csv',
 'C:/Users/sortiz/Onedrive - Cinedigm/Databases/SVOD/Amazon SVOD DB/Amazon Subs/Screambox/2020/subs_m_US_Screambox_2020-05.csv',
 'C:/Users/sortiz/Onedrive - Cinedigm/Databases/SVOD/Amazon SVOD DB/Amazon Subs/Screambox/2020/subs_m_US_Screambox_2020-06.csv',
 'C:/Users/sortiz/Onedrive - Cinedigm/Databases/SVOD/Amazon SVOD DB/Amazon Subs/Screambox/2020/subs_m_US_Screambox_2020-07.csv',
 'C:/Users/sortiz/Onedrive - Cinedigm/Databases/SVOD/Amazon SVOD DB/Amazon Subs/Screambox/2020/subs_m_US_Screambox_2020-08.csv',
 'C:/Users/sortiz/Onedrive - Cinedigm/Databases/SVOD/Amazon SVOD DB/Amazon Subs/Screambox/2020/subs_m_US_Screambox_2020-09.csv',
 'C:/Users/sortiz/Onedrive - Cinedigm/Databases/SVOD/Amazon SVOD DB/Amazon Subs/Screambox/2020/subs_m_US_Screambox_2020-10.csv',
 'C:/Users/sortiz/Onedrive - Cinedigm/Databases/SVOD/Amazon SVOD DB/Amazon Subs/Screambox/2020/su

In [9]:
rfgf

NameError: name 'rfgf' is not defined

In [ ]:
dfsubs = pd.concat([pd.read_csv(file) for file in l]).reset_index()
dfsubs= dfsubs.loc[dfsubs['Subscription Category']== 'HO',('Report Start','Subscription','Billing Amount','Active Subscriptions')]

In [ ]:
dfFsubs = pd.concat([pd.read_csv(file) for file in lF]).reset_index()
# Fandor_last= pd.read_csv(SFilepath + 'Fandor/US/2022/subs_m_US_Fandor_2022-04.csv')
# dfFsubs = pd.concat([dfFsubs, Fandor_last])

dfFsubs= dfFsubs.loc[dfFsubs['Subscription Category']== 'HO',('Report Start','Subscription','Billing Amount','Active Subscriptions')]
dfFsubs['Subscription']='Fandor'
dfFsubs= dfFsubs.groupby(by=['Report Start','Subscription']).agg(sum).reset_index()
dfFsubs['Billing Amount']= 3.99

In [ ]:
Amz_subs= pd.concat([dfsubs,dfFsubs]).reset_index()
Amz_subs.rename(columns={'Active Subscriptions':'Amazon Paid Subs'}, inplace=True)
Amz_subs['Report Start']= pd.to_datetime(Amz_subs['Report Start'])
Amz_subs.loc[Amz_subs['Subscription']=='Dove Channel', 'Subscription'] = 'Dove'

In [ ]:
Amz_subs.head(200)

# Roku Subs (Importing, Transforming)

In [ ]:
Rok_subs= pd.read_csv('C://Users/sortiz/OneDrive - Cinedigm/Databases/SVOD/Roku SVOD DB/Roku Subs.csv')

In [ ]:
Rok_subs['Date Key']=pd.to_datetime(Rok_subs['Date Key'])
Rok_subs.rename(columns={'Date Key':'Report Start','Provider Product ID':'Subscription','Total Active Paid End Users':'Roku Paid Subs'}, inplace=True)

In [ ]:
Rok_subs.head(10)

# Merge Subs into one Table 'Subs'

In [ ]:
Subs= pd.merge(Amz_subs, Rok_subs.loc[:,('Report Start','Subscription','Roku Paid Subs')], on= ['Report Start', 'Subscription'], how= 'left')

In [ ]:
Subs['Year']= Subs['Report Start'].dt.year
Subs['Month(num)']= Subs['Report Start'].dt.month
Subs['Total Paid Subs']=Subs['Amazon Paid Subs']+ Subs['Roku Paid Subs']
Subs['Revenue']= Subs['Billing Amount']* Subs['Total Paid Subs']

In [ ]:
Cond=[
    Subs['Month(num)']==1, Subs['Month(num)']==2, Subs['Month(num)']==3, Subs['Month(num)']==4,
    Subs['Month(num)']==5, Subs['Month(num)']==6, Subs['Month(num)']==7, Subs['Month(num)']==8,
    Subs['Month(num)']==9, Subs['Month(num)']==10, Subs['Month(num)']==11, Subs['Month(num)']==12
]
Months_names = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']

Subs['Month'] = np.select(Cond,Months_names, default=np.nan)

In [ ]:
Subs=Subs.loc[:,('Year','Month(num)','Month','Subscription','Amazon Paid Subs','Roku Paid Subs','Billing Amount','Total Paid Subs','Revenue')]
Subs= Subs.sort_values(by=['Subscription','Year','Month(num)'])

In [ ]:
Subs.head(200)

# Amazon Content Data (Importing, Transforming)

In [ ]:
CFilepath='C:/Users/sortiz/Onedrive - Cinedigm/Databases/SVOD/Amazon SVOD DB/Amazon Content/'

Channels = ['Screambox', 'Docurama', 'CONtv', 'Dove Channel']
CountryF = ['US','UK']

In [ ]:
#importing Content for all subs except Fandor

cont_lists = {'2020-04':'2020','2020-05':'2020','2020-06':'2020','2020-07':'2020','2020-08':'2020','2020-09':'2020','2020-10':'2020','2020-11':'2020','2020-12':'2020',
         '2021-01':'2021','2021-02':'2021','2021-03':'2021','2021-04':'2021','2021-05':'2021','2021-06':'2021','2021-07':'2021','2021-08':'2021','2021-09':'2021','2021-10':'2021','2021-11':'2021','2021-12':'2021',
         '2022-01':'2022','2022-02':'2022','2022-03':'2022','2022-04':'2022','2022-05':'2022','2022-06':'2022','2022-07':'2022'}
cl=[]

for i in Channels: 
    for key,value in cont_lists.items():
        cl = cl + [CFilepath + i +'/' + value + '/contusg_m_US_' + i + '_' + key + '.csv']
        

In [ ]:
#importing Content for Fandor (Since UK data is always late)

cont_listsF = {'2020-04':'2020','2020-05':'2020','2020-06':'2020','2020-07':'2020','2020-08':'2020','2020-09':'2020','2020-10':'2020','2020-11':'2020','2020-12':'2020',
         '2021-01':'2021','2021-02':'2021','2021-03':'2021','2021-04':'2021','2021-05':'2021','2021-06':'2021','2021-07':'2021','2021-08':'2021','2021-09':'2021','2021-10':'2021','2021-11':'2021','2021-12':'2021',
         '2022-01':'2022','2022-02':'2022','2022-03':'2022','2022-04':'2022','2022-05':'2022','2022-06':'2022','2022-07':'2022'}
clF=[]

for i in CountryF: 
    for key,value in cont_listsF.items():
        clF = clF + [CFilepath + 'Fandor/' + i +'/' + value + '/contusg_m_' + i + '_' + 'Fandor' + '_' + key + '.csv']
        

In [ ]:
dfcont = pd.concat([pd.read_csv(file) for file in cl]).reset_index()

In [ ]:
dfFcont = pd.concat([pd.read_csv(file) for file in clF]).reset_index()
#Fandor_last= pd.read_csv(CFilepath + 'Fandor/US/2022/contusg_m_US_Fandor_2022-04.csv')
#dfFcont = pd.concat([dfFcont, Fandor_last])

In [ ]:
dfFcont['Series or Movie Title']= dfFcont['Series or Movie Title'].fillna(dfFcont['Season or Movie Title'])

In [ ]:
dfFcont= dfFcont.drop(columns=['Report Date','Season or Movie Title'])

In [ ]:
Amz_cont=pd.concat([dfcont,dfFcont])

In [ ]:
Amz_cont['Report Start']= pd.to_datetime(Amz_cont['Report Start'])
Amz_cont.loc[Amz_cont['Subscription']=='Dove Channel','Subscription']='Dove'
Amz_cont['Year']= Amz_cont['Report Start'].dt.year
Amz_cont['Month']= Amz_cont['Report Start'].dt.month
Amz_cont= Amz_cont.loc[:,('Year','Month','Subscription','Vendor SKU','Content Type','Series or Movie Title','Total Minutes Streamed','Total Number of Streams')]
Amz_cont.rename(columns={'Vendor SKU':'ID','Content Type':'TV/MOVIE','Series or Movie Title':'Title'}, inplace=True)

#There is multiple titles for the same Year/Month because its broken down by Payment type, as in paid subs vs free trial etc.

Amz_cont= Amz_cont.groupby(by=['Year','Month','Subscription','ID','TV/MOVIE','Title'])[['Total Minutes Streamed','Total Number of Streams']].agg(sum).reset_index()

In [ ]:
Amz_cont.loc[Amz_cont['Title'].str.contains('false p',case=False)]

In [ ]:
Amz_cont['ID']= Amz_cont['ID'].astype(str) 
Amz_cont['Title']= Amz_cont['Title'].astype(str) 
Amz_cont['Subscription']= Amz_cont['Subscription'].astype(str) 
Amz_cont['TV/MOVIE']= Amz_cont['TV/MOVIE'].astype(str) 

In [ ]:
# Amz_cont.loc[Amz_cont['Title'].str.contains('the collector',case=False),'ID']='883476025872'
# Amz_cont.loc[Amz_cont['Title'].str.contains('A Turtle\'s Tale',case=False),'ID']='883476075594'

In [ ]:
ACo=Amz_cont.loc[Amz_cont['ID'].str[:3]=='008'].drop_duplicates()

In [ ]:
ACo['ID2']= ACo['ID'].str[2:]

In [ ]:
ACo

In [ ]:
Amz_cont=pd.merge(Amz_cont,ACo.loc[:,('ID','ID2')], on='ID', how='left')

In [ ]:
Amz_cont['ID']=Amz_cont['ID2'].fillna(Amz_cont['ID'])
Amz_cont.drop_duplicates(inplace=True)

In [ ]:
Amz_cont

In [ ]:
#small fix

Amz_cont.loc[Amz_cont['Title']=='Sue Thomas: F.B.Eye','ID']='CEG10019302'

In [ ]:
Amz_cont.loc[Amz_cont['Title'].str.contains('Sue Thomas',case=False)]

In [ ]:
Amz_cont= Amz_cont.groupby(by=['Year','Month','Subscription','ID','TV/MOVIE','Title'])[['Total Minutes Streamed','Total Number of Streams']].agg(sum).reset_index()

In [ ]:
Amz_cont= Amz_cont.loc[:,('Year','Month','Subscription','ID','TV/MOVIE','Title','Total Minutes Streamed','Total Number of Streams')]

In [ ]:
Amz_cont

In [ ]:
Amz_cont.loc[Amz_cont['Title']=='11/11/11','Title']='11_11_11'

# Search

In [ ]:
Amz_cont.loc[(Amz_cont['ID'].str.contains('sue',case=False)),('Subscription','Title','ID','TV/MOVIE')]

# Roku Content Data (Importing, Transforming)

In [ ]:
RC_Filepath='C:/Users/sortiz/Onedrive - Cinedigm/Databases/SVOD/Roku SVOD DB/Roku Content/'

ContR_0= pd.read_excel(RC_Filepath + 'Roku SVOD 04-2020_02-2022.xlsx', sheet_name='Hours')
ContR_1= pd.read_excel(RC_Filepath + 'Roku SVOD 03-2022.xlsx', sheet_name='Hours')
ContR_2= pd.read_excel(RC_Filepath + 'Roku SVOD 04-2022.xlsx', sheet_name='Hours')
ContR_3= pd.read_excel(RC_Filepath + 'Roku SVOD 05-2022.xlsx', sheet_name='Hours')
ContR_4= pd.read_excel(RC_Filepath + 'Roku SVOD 06-2022.xlsx', sheet_name='Hours')
ContR_5= pd.read_excel(RC_Filepath + 'Roku SVOD 07-2022.xlsx', sheet_name='Hours')

ContR_0S= pd.read_excel(RC_Filepath + 'Roku SVOD 04-2020_02-2022.xlsx', sheet_name='Streams')
ContR_1S= pd.read_excel(RC_Filepath + 'Roku SVOD 03-2022.xlsx', sheet_name='Streams')
ContR_2S= pd.read_excel(RC_Filepath + 'Roku SVOD 04-2022.xlsx', sheet_name='Streams')
ContR_3S= pd.read_excel(RC_Filepath + 'Roku SVOD 05-2022.xlsx', sheet_name='Streams')
ContR_4S= pd.read_excel(RC_Filepath + 'Roku SVOD 06-2022.xlsx', sheet_name='Streams')
ContR_5S= pd.read_excel(RC_Filepath + 'Roku SVOD 07-2022.xlsx', sheet_name='Streams')

In [ ]:
glossaryAR= pd.read_excel('C:/Users/sortiz/OneDrive - Cinedigm/Databases/SVOD/Roku SVOD DB/Roku_Amazon Glossary.xlsx', sheet_name='Roku_Amazon Glossary')

In [ ]:
# 04-2020 to 02-2022 Hours
ContR_0= ContR_0.loc[:, ~ContR_0.columns.isin(['Content Title','Season Number','Episode Number'])]
ContR_0= ContR_0.melt(id_vars=['Channel','TV/MOVIE','Title'], var_name='Date', value_name='Total Hours Viewed')

# 03-2022 Hours
ContR_1= ContR_1.loc[:, ~ContR_1.columns.isin(['Content Title','Season Number','Episode Number'])]
ContR_1= ContR_1.melt(id_vars=['Channel','TV/MOVIE','Title'], var_name='Date', value_name='Total Hours Viewed')

# 04-2022 Hours
ContR_2= ContR_2.loc[:, ~ContR_2.columns.isin(['Content Title','Season Number','Episode Number'])]
ContR_2= ContR_2.melt(id_vars=['Channel','TV/MOVIE','Title'], var_name='Date', value_name='Total Hours Viewed')

# 05-2022 Hours
ContR_3= ContR_3.loc[:, ~ContR_3.columns.isin(['Content Title','Season Number','Episode Number'])]
ContR_3= ContR_3.melt(id_vars=['Channel','TV/MOVIE','Title'], var_name='Date', value_name='Total Hours Viewed')

# 06-2022 Hours
ContR_4= ContR_4.loc[:, ~ContR_4.columns.isin(['Content Title','Season Number','Episode Number'])]
ContR_4= ContR_4.melt(id_vars=['Channel','TV/MOVIE','Title'], var_name='Date', value_name='Total Hours Viewed')

# 07-2022 Hours
ContR_5= ContR_5.loc[:, ~ContR_5.columns.isin(['Content Title','Season Number','Episode Number'])]
ContR_5= ContR_5.melt(id_vars=['Channel','TV/MOVIE','Title'], var_name='Date', value_name='Total Hours Viewed')

# STREAMS
# 04-2020 to 02-2022 Streams
ContR_0S= ContR_0S.loc[:, ~ContR_0S.columns.isin(['Content Title','Season Number','Episode Number'])]
ContR_0S= ContR_0S.melt(id_vars=['Channel','TV/MOVIE','Title'], var_name='Date', value_name='Total Number of Streams')

# 03-2022 Streams
ContR_1S= ContR_1S.loc[:, ~ContR_1S.columns.isin(['Content Title','Season Number','Episode Number'])]
ContR_1S= ContR_1S.melt(id_vars=['Channel','TV/MOVIE','Title'], var_name='Date', value_name='Total Number of Streams')

# 04-2022 Streams
ContR_2S= ContR_2S.loc[:, ~ContR_2S.columns.isin(['Content Title','Season Number','Episode Number'])]
ContR_2S= ContR_2S.melt(id_vars=['Channel','TV/MOVIE','Title'], var_name='Date', value_name='Total Number of Streams')

# 05-2022 Streams
ContR_3S= ContR_3S.loc[:, ~ContR_3S.columns.isin(['Content Title','Season Number','Episode Number'])]
ContR_3S= ContR_3S.melt(id_vars=['Channel','TV/MOVIE','Title'], var_name='Date', value_name='Total Number of Streams')

# 06-2022 Streams
ContR_4S= ContR_4S.loc[:, ~ContR_4S.columns.isin(['Content Title','Season Number','Episode Number'])]
ContR_4S= ContR_4S.melt(id_vars=['Channel','TV/MOVIE','Title'], var_name='Date', value_name='Total Number of Streams')

# 07-2022 Streams
ContR_5S= ContR_5S.loc[:, ~ContR_5S.columns.isin(['Content Title','Season Number','Episode Number'])]
ContR_5S= ContR_5S.melt(id_vars=['Channel','TV/MOVIE','Title'], var_name='Date', value_name='Total Number of Streams')

In [ ]:
Rok_Hrs= pd.concat([ContR_0,ContR_1,ContR_2,ContR_3,ContR_4,ContR_5]).reset_index()
Rok_Str= pd.concat([ContR_0S,ContR_1S,ContR_2S,ContR_3S,ContR_4S,ContR_5S]).reset_index()

In [ ]:
#Hours
Rok_Hrs['Date']=pd.to_datetime(Rok_Hrs['Date'])
Rok_Hrs['Year']=Rok_Hrs['Date'].dt.year
Rok_Hrs['Month']=Rok_Hrs['Date'].dt.month
Rok_Hrs['Total Minutes Viewed']= Rok_Hrs['Total Hours Viewed']*60
Rok_Hrs.drop(columns=['Total Hours Viewed'], inplace=True)
Rok_Hrs.rename(columns={'Channel':'Subscription','Total Minutes Viewed':'Roku Tot Min Viewed'},inplace=True)
Rok_Hrs= Rok_Hrs.groupby(by=['Date','Year','Month','Subscription','TV/MOVIE','Title'])[['Roku Tot Min Viewed']].agg(sum).reset_index()

#Streams
Rok_Str['Date']=pd.to_datetime(Rok_Str['Date'])
Rok_Str['Year']=Rok_Str['Date'].dt.year
Rok_Str['Month']=Rok_Str['Date'].dt.month
Rok_Str.rename(columns={'Channel':'Subscription','Total Number of Streams':'Roku Streams'},inplace=True)
Rok_Str= Rok_Str.groupby(by=['Date','Year','Month','Subscription','TV/MOVIE','Title'])[['Roku Streams']].agg(sum).reset_index()

In [ ]:
#Merge Minutes And Streams
ContR= pd.merge(Rok_Hrs, Rok_Str, on=('Date','Year','Month','Subscription','TV/MOVIE','Title'), how='left')

In [ ]:
ContR.info()

In [ ]:
ContR.loc[ContR['Title'].str.contains('false positive',case=False)]

In [ ]:
ContR.fillna(0, inplace=True)

In [ ]:
ContR.loc[ContR.isna().any(axis=1)]

In [ ]:
ContR.loc[ContR['Title'].str.contains('Masters of ', na=True),:].tail(5)

In [ ]:
#Small Fixes
ContR.loc[ContR['Title'].str.contains('Masters of Horror'),'TV/MOVIE']='TV'
ContR.loc[ContR['Title'].str.contains('C.O.P.S.'),'Title']='C.O.P.S'

glossaryAR.rename(columns={'Channel':'Subscription','Roku Title':'Title'}, inplace=True)
ContR=pd.merge(ContR,glossaryAR, on=['Subscription','TV/MOVIE','Title'], how='left')
ContR=ContR.loc[(ContR['Roku Streams']!= 0) | (ContR['Roku Tot Min Viewed']!= 0),:]
ContR=ContR[ContR['Roku Streams'].notna()]

In [ ]:
ContR.drop_duplicates(inplace=True)

In [ ]:
ContR['ID']= ContR['ID'].astype(str) 
ContR['Title']= ContR['Title'].astype(str) 
ContR['Subscription']= ContR['Subscription'].astype(str) 
ContR['TV/MOVIE']= ContR['TV/MOVIE'].astype(str) 

In [ ]:
#Small Fix
ContR.loc[ContR['Title']=='Sue Thomas F.B.Eye','Title']='Sue Thomas: F.B.Eye'
ContR.loc[ContR['Title']=='Sue Thomas, l\'oeil du FBI','Title']='Sue Thomas: F.B.Eye'
ContR.loc[ContR['Title']=='Sue Thomas: F.B.Eye','ID']='CEG10019302'

In [ ]:
ContR.columns

In [ ]:
ContR

In [ ]:
ContR.loc[ContR['Title']=='11/11/11','Title']='11_11_11'

# Search

In [ ]:
ContR.loc[ContR['ID'].str.contains('nan', case=False),('Subscription','TV/MOVIE','Title','ID')].drop_duplicates()

# Merge Both AMAZON and ROKU

In [ ]:
Content= pd.merge(Amz_cont, ContR.loc[:,('Year','Month','Subscription','TV/MOVIE','ID','Title','Roku Tot Min Viewed','Roku Streams')], on=['Subscription','Year','Month','TV/MOVIE','ID'], how='outer')

In [ ]:
Content

In [ ]:
Content.loc[Content['ID']=='CEG10019302',:]

In [ ]:
Content.rename(columns={'Total Minutes Streamed':'Amzn Min Streamed','Total Number of Streams':'Amzn Streams','Roku Tot Min Viewed':'Roku Min Streamed'}, inplace = True)
Content['Title']=Content['Title_x'].fillna(Content['Title_y'])

Content['Amzn Min Streamed']= Content['Amzn Min Streamed'].fillna(0)
Content['Amzn Streams']= Content['Amzn Streams'].fillna(0)
Content['Roku Min Streamed']= Content['Roku Min Streamed'].fillna(0)
Content['Roku Streams']= Content['Roku Streams'].fillna(0)


Content['Total Min Streamed']= Content['Amzn Min Streamed']+ Content['Roku Min Streamed']
Content['Total Streams']= Content['Amzn Streams']+ Content['Roku Streams']

Content= Content.loc[:,('Year','Month','Subscription','ID','TV/MOVIE','Title','Amzn Min Streamed','Amzn Streams','Roku Min Streamed','Roku Streams','Total Min Streamed','Total Streams')]
Content=Content.sort_values(by=['Subscription','Title']).reset_index(drop=True)

In [ ]:
Content

In [ ]:
#Content['TITLE']=Content.groupby('ID')['Title'].transform('first')
Content['ID']= Content['ID'].astype(str) 
Content['Title']= Content['Title'].astype(str) 
Content['Subscription']= Content['Subscription'].astype(str) 
Content['TV/MOVIE']= Content['TV/MOVIE'].astype(str) 

In [ ]:
#Content['Title']= Content.groupby('ID')['Title'].transform('first')

In [ ]:
Content.loc[Content['ID']=='CEG10019302',:]

In [ ]:
Content.loc[Content['Title'].str.contains('inspector', case=False)]

# Content without ID, Groups the episodes together

In [ ]:
# Small Fix

Content.loc[Content['ID']=='1000000002887','Title']='Masters of Horror'

In [ ]:
Content2=Content.loc[:,:]
Content2=Content2.groupby(by=['Subscription','Year','Month','TV/MOVIE','ID']).agg(sum).reset_index()

In [ ]:
Content2.loc[Content2['ID']=='883476075594',:]

In [ ]:
Idt=Content.loc[:,('Subscription','ID','Title','TV/MOVIE')]

In [ ]:
Idt.drop_duplicates(inplace=True)

In [ ]:
Idt.drop_duplicates(subset=('Subscription','TV/MOVIE','ID'), keep='first', inplace=True)

In [ ]:
Idt

In [ ]:
Content2=pd.merge(Content2,Idt, on=('Subscription','TV/MOVIE','ID'), how='left')

In [ ]:
Content2=Content2.groupby(by=['Year','Month','Subscription','TV/MOVIE','Title']).agg(sum).reset_index()

Idt2=Content.loc[:,('Subscription','ID','Title','TV/MOVIE')]
Idt2.drop_duplicates(inplace=True)
Idt2.drop_duplicates(subset=('Subscription','TV/MOVIE','Title'), keep='last', inplace=True)

Content2=pd.merge(Content2,Idt2, on=('Subscription','TV/MOVIE','Title'), how='left')

In [ ]:
Idt2

In [ ]:
Content2

In [ ]:
Cond1=[
    Content2['Month']==1, Content2['Month']==2, Content2['Month']==3, Content2['Month']==4,
    Content2['Month']==5, Content2['Month']==6, Content2['Month']==7, Content2['Month']==8,
    Content2['Month']==9, Content2['Month']==10, Content2['Month']==11, Content2['Month']==12
]
Months_names1 = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']

Content2['Month Abv'] = np.select(Cond1,Months_names1, default=np.nan)

In [ ]:
Con2= Content2.loc[:,:]

In [ ]:
Con2.loc[Con2['Title'].str.contains('sue thomas', case=False),:]
#Con2.loc[Con2['ID']=='883476075594',:]

In [ ]:
Con2.loc[Con2['ID'].str[:2]=='00']

In [ ]:
Content2['Amzn Min/Stream']= Content2['Amzn Min Streamed']/ Content2['Amzn Streams']
Content2['Roku Min/Stream']= Content2['Roku Min Streamed']/ Content2['Roku Streams']
Content2['Min/Stream']= Content2['Total Min Streamed']/ Content2['Total Streams']

# Rev, Percentile, Total Channel Minutes

In [ ]:
#Total Minutes per Month
TotM=Content2.groupby(by=['Subscription','Year','Month'])[['Total Min Streamed']].agg(sum).reset_index()

#Total Rev
RevM=Subs.loc[:,('Year','Month(num)','Subscription','Revenue')]

#Merge
TotMRevM=pd.merge(TotM,RevM, left_on=('Subscription','Year','Month'), right_on=('Subscription','Year','Month(num)'), how='left')

In [ ]:
TotMRevM.rename(columns={'Total Min Streamed':'Tot Sub Month Min','Revenue':'Tot Sub Month Rev'}, inplace=True)
TotMRevM= TotMRevM.loc[:,('Subscription','Year','Month','Tot Sub Month Min','Tot Sub Month Rev')]

In [ ]:
TotMRevM

In [ ]:
Content2= pd.merge(Content2, TotMRevM, on=('Subscription','Year','Month'), how='left')

In [ ]:
Content2['PerTM']=Content2['Total Min Streamed']/Content2['Tot Sub Month Min']
Content2['Content Month Rev']=Content2['PerTM']*Content2['Tot Sub Month Rev']
Content2['Percentile Rank (Min)']=Content2.groupby(by=['Subscription','Year','Month'])['Total Min Streamed'].rank(pct=True)

In [ ]:
Content2= Content2.loc[:,('Subscription','Year','Month','Month Abv','ID','TV/MOVIE','Title',
                          'Amzn Min Streamed','Amzn Streams','Amzn Min/Stream','Roku Min Streamed','Roku Streams','Roku Min/Stream',
                          'Total Min Streamed','Percentile Rank (Min)','Total Streams','Min/Stream','Content Month Rev')]

In [ ]:
Content2.head(20)

# Trend Month by Month

In [ ]:
Content3= Content2.loc[:,:]

In [ ]:
Content3['Date']= Content3['Month Abv']+'-'+Content3['Year'].astype(str)

In [ ]:
Content2

In [ ]:
CMSS= Content3.pivot(index=['Subscription','ID','TV/MOVIE','Title'], columns='Date', values=['Total Min Streamed']).reset_index()
CMSSM= Content3.pivot(index=['Subscription','ID','TV/MOVIE','Title'], columns='Date', values=['Min/Stream']).reset_index()
RPP= Content3.pivot(index=['Subscription','ID','TV/MOVIE','Title'], columns='Date', values=['Percentile Rank (Min)']).reset_index()

In [ ]:
CMSS['Metric']='Minutes Streamed'
CMSSM['Metric']='Min/Stream'
RPP['Metric']='Percentile Rank (Min)'

In [ ]:
CMSS

In [ ]:
ColumnD=['Apr-20','Apr-21','Apr-22','Aug-20','Aug-21','Dec-20','Dec-21','Feb-21','Feb-22','Jan-21','Jan-22','Jul-20','Jul-21','Jul-22',
         'Jun-20','Jun-21','Jun-22','Mar-21','Mar-22','May-20','May-21','May-22','Nov-20','Nov-21','Oct-20','Oct-21','Sep-20','Sep-21']

CMSS.columns=['Subscription','ID','TV/MOVIE','Title']+ColumnD+['Metric']
CMSSM.columns=['Subscription','ID','TV/MOVIE','Title']+ColumnD+['Metric']
RPP.columns=['Subscription','ID','TV/MOVIE','Title']+ColumnD+['Metric']

CMSS=CMSS.loc[:,('Subscription','ID','TV/MOVIE','Title','Apr-20','May-20','Jun-20','Jul-20','Aug-20','Sep-20','Oct-20','Nov-20','Dec-20',
                 'Jan-21','Feb-21','Mar-21','Apr-21','May-21','Jun-21','Jul-21','Aug-21','Sep-21','Oct-21','Nov-21','Dec-21',
                 'Jan-22','Feb-22','Mar-22','Apr-22','May-22','Jun-22','Jul-22','Metric')]

CMSSM=CMSSM.loc[:,('Subscription','ID','TV/MOVIE','Title','Apr-20','May-20','Jun-20','Jul-20','Aug-20','Sep-20','Oct-20','Nov-20','Dec-20',
                   'Jan-21','Feb-21','Mar-21','Apr-21','May-21','Jun-21','Jul-21','Aug-21','Sep-21','Oct-21','Nov-21','Dec-21',
                   'Jan-22','Feb-22','Mar-22','Apr-22','May-22','Jun-22','Jul-22','Metric')]

RPP=RPP.loc[:,('Subscription','ID','TV/MOVIE','Title','Apr-20','May-20','Jun-20','Jul-20','Aug-20','Sep-20','Oct-20','Nov-20','Dec-20',
               'Jan-21','Feb-21','Mar-21','Apr-21','May-21','Jun-21','Jul-21','Aug-21','Sep-21','Oct-21','Nov-21','Dec-21',
               'Jan-22','Feb-22','Mar-22','Apr-22','May-22','Jun-22','Jul-22','Metric')]

In [ ]:
CM=pd.concat([CMSS,CMSSM,RPP])

In [ ]:
#Average
CM['Average']=CM.loc[:,'Apr-20':'Jul-22'].mean(axis=1)

#Average Monthly Difference
CM['F']=CM.loc[:,'Apr-20':'Jul-22'].fillna(method='bfill',axis=1).iloc[:,0]
CM['L']=CM.loc[:,'Apr-20':'Jul-22'].fillna(method='ffill',axis=1).iloc[:,-1]
CM['C']=CM.loc[:,'Apr-20':'Jul-22'].count(axis=1)

CM['Trend']= (CM['L']-CM['F'])/(CM['C']-1)

In [ ]:
CM=CM.loc[:,('Subscription','ID','TV/MOVIE','Title','Apr-20','May-20','Jun-20','Jul-20','Aug-20','Sep-20','Oct-20','Nov-20','Dec-20',
             'Jan-21','Feb-21','Mar-21','Apr-21','May-21','Jun-21','Jul-21','Aug-21','Sep-21','Oct-21','Nov-21','Dec-21',
             'Jan-22','Feb-22','Mar-22','Apr-22','May-22','Jun-22','Jul-22','Metric','Average','Trend')]

In [ ]:
CM.head(-100)

# Adding Total Minutes and Streams to Sub table

In [ ]:
C3= Content2.loc[:,('Subscription','Year','Month','Total Min Streamed','Total Streams','Amzn Min Streamed','Amzn Streams','Roku Min Streamed','Roku Streams')]
C3= C3.groupby(by=['Subscription','Year','Month',])[['Total Min Streamed','Total Streams','Amzn Min Streamed','Amzn Streams','Roku Min Streamed','Roku Streams']].agg(sum).reset_index()
Subs= pd.merge(Subs,C3, left_on=('Subscription','Year','Month(num)'), right_on=('Subscription','Year','Month'), how='left')
Subs.rename(columns={'Month_x':'Month'}, inplace=True)
# Subs= Subs.loc[:,('Subscription','Year','Month(num)','Month','Amazon Paid Subs','Amzn Min Streamed','Amzn Streams',
#                  'Roku Paid Subs','Roku Min Streamed','Roku Streams','Billing Amount','Total Paid Subs','Revenue','Total Min Streamed','Total Streams')]

Subs['Amzn Min/Stream']= Subs['Amzn Min Streamed']/Subs['Amzn Streams']
Subs['Roku Min/Stream']= Subs['Roku Min Streamed']/Subs['Roku Streams']
Subs['Avg Min/Stream']= Subs['Total Min Streamed']/Subs['Total Streams']

Subs['Amzn Min/Sub']= Subs['Amzn Min Streamed']/Subs['Amazon Paid Subs']
Subs['Roku Min/Sub']= Subs['Roku Min Streamed']/Subs['Roku Paid Subs']
Subs['Min/Sub']= Subs['Total Min Streamed']/Subs['Total Paid Subs']

In [ ]:
Content2

In [ ]:
C3

In [ ]:
Subs['Amzn Rev']=Subs['Amazon Paid Subs']*Subs['Billing Amount']
Subs['Roku Rev']=Subs['Roku Paid Subs']*Subs['Billing Amount']

Subs= Subs.loc[:,('Subscription','Year','Month(num)','Month','Billing Amount',
                  'Amazon Paid Subs','Amzn Rev','Amzn Min Streamed','Amzn Streams','Amzn Min/Stream','Amzn Min/Sub',
                  'Roku Paid Subs','Roku Rev','Roku Min Streamed','Roku Streams','Roku Min/Stream','Roku Min/Sub',
                  'Total Paid Subs','Revenue','Total Min Streamed','Total Streams','Avg Min/Stream','Min/Sub')]

In [ ]:
Subs.head(200)

# REVENUE

In [ ]:
Rev= Content2.loc[:,('Subscription','Year','Month','ID','Date','TV/MOVIE','Title','Content Month Rev')]

In [ ]:
Rev1= Rev.pivot(index=['Subscription','ID','TV/MOVIE','Title'], columns='Date', values=['Content Month Rev']).reset_index()

In [ ]:
Rev1.columns=['Subscription','ID','TV/MOVIE','Title']+ColumnD

Rev1=Rev1.loc[:,('Subscription','ID','TV/MOVIE','Title','Apr-20','May-20','Jun-20','Jul-20','Aug-20','Sep-20','Oct-20','Nov-20','Dec-20',
                 'Jan-21','Feb-21','Mar-21','Apr-21','May-21','Jun-21','Jul-21','Aug-21','Sep-21','Oct-21','Nov-21','Dec-21',
                 'Jan-22','Feb-22','Mar-22','Apr-22','May-22','Jun-22','Jul-22')]

#Average
Rev1['Average']=Rev1.loc[:,'Apr-20':'Jul-22'].mean(axis=1)

#Average Monthly Difference
Rev1['F']=Rev1.loc[:,'Apr-20':'Jul-22'].fillna(method='bfill',axis=1).iloc[:,0]
Rev1['L']=Rev1.loc[:,'Apr-20':'Jul-22'].fillna(method='ffill',axis=1).iloc[:,-1]
Rev1['C']=Rev1.loc[:,'Apr-20':'Jul-22'].count(axis=1)

Rev1['Trend']= (Rev1['L']-Rev1['F'])/(Rev1['C']-1)



Rev1=Rev1.loc[:,('Subscription','ID','TV/MOVIE','Title','Apr-20','May-20','Jun-20','Jul-20','Aug-20','Sep-20','Oct-20','Nov-20','Dec-20',
                 'Jan-21','Feb-21','Mar-21','Apr-21','May-21','Jun-21','Jul-21','Aug-21','Sep-21',
                 'Oct-21','Nov-21','Dec-21','Jan-22','Feb-22','Mar-22','Apr-22','May-22','Jun-22','Jul-22','Average','Trend')]

In [ ]:
Rev1.rename(columns={'Average':'Roku+Amzn monthly Rev Avg'},inplace=True)
Rev1['All Platforms monthly Rev Avg (10/8 estimate)']= Rev1['Roku+Amzn monthly Rev Avg']*(10/8)

In [ ]:
TotMRevM['Date']= TotMRevM['Month'].astype(str)+'-'+TotMRevM['Year'].astype(str)
TotMRevM= TotMRevM.pivot(index=['Subscription'], columns='Date', values=['Tot Sub Month Rev']).reset_index()

In [ ]:
TotMRevM

In [ ]:
TotMRevM.columns=['Subscription','1-2021','1-2022','10-2020','10-2021','11-2020','11-2021','12-2020','12-2021','2-2021','2-2022','3-2021','3-2022',
                  '4-2020','4-2021','4-2022','5-2020','5-2021','5-2022','6-2020','6-2021','6-2022','7-2020','7-2021','7-2022','8-2020','8-2021','9-2020','9-2021']
TotMRevM=TotMRevM.loc[:,('Subscription','4-2020','5-2020','6-2020','7-2020','8-2020','9-2020','10-2020','11-2020','12-2020',
                         '1-2021','2-2021','3-2021','4-2021','5-2021','6-2021','7-2021','8-2021','9-2021','10-2021','11-2021','12-2021',
                         '1-2022','2-2022','3-2022','4-2022','5-2022','6-2022','7-2022')]

In [ ]:
# TotMRevM.insert(1,'4-2020',[136466.52,52210.37,132389.69,178532.55,29715.45])
# TotMRevM.insert(2,'5-2020',[149580.24,56756.26,147564.28,193175.85,39071.7])
# TotMRevM.insert(3,'6-2020',[175413.47,65089.56,167938.45,224461.44,47884.04])
# TotMRevM.insert(4,'7-2020',[186725.8,68183.36,173996.31,224058.45,49321.16])
# TotMRevM.insert(5,'8-2020',[186496.26,69326.07,184415.43,222430.53,48732.34])
# TotMRevM.insert(6,'9-2020',[184450.36,74226.25,185143.97,222175.17,49430.94])
# TotMRevM.insert(7,'10-2020',[187010.23,79745.19,187673.9,226815.54,55149.48])
# TotMRevM.insert(8,'11-2020',[193442.34,79790.1,189999.24,229967.64,54076.63])
# TotMRevM.insert(9,'12-2020',[193846.53,79770.14,194155.91,231846.93,58956.85])

In [ ]:
TotMRevM.loc[-1]=['Number',1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28]

In [ ]:
TotMRevM

In [ ]:
def linear_regression_int(x, y):     
    N = len(x)
    x_mean = x.mean()
    y_mean = y.mean()
    
    B1_num = ((x - x_mean) * (y - y_mean)).sum()
    B1_den = ((x - x_mean)**2).sum()
    Slope = B1_num / B1_den
    
    B0 = y_mean - (Slope*x_mean)
    
    
    return (B0)

In [ ]:
def linear_regression_slope(x, y):     
    N = len(x)
    x_mean = x.mean()
    y_mean = y.mean()
    
    B1_num = ((x - x_mean) * (y - y_mean)).sum()
    B1_den = ((x - x_mean)**2).sum()
    
    
    Slope = B1_num / B1_den
    
    B0 = y_mean - (Slope*x_mean)
    
    
    return (Slope)

In [ ]:
TotMRevM

In [ ]:
RevP= TotMRevM.set_index('Subscription').T
RevP.reset_index(inplace=True)
RevP.columns=['Date','CONtv','Docurama','Dove','Fandor','Screambox','Number']

In [ ]:
RevP

In [ ]:
linear_regression_slope(RevP['Number'],RevP['Fandor'])

In [ ]:
RevPrj= pd.DataFrame([
    ['CONtv',linear_regression_int(RevP['Number'],RevP['CONtv']),linear_regression_slope(RevP['Number'],RevP['CONtv'])],
    ['Docurama',linear_regression_int(RevP['Number'],RevP['Docurama']),linear_regression_slope(RevP['Number'],RevP['Docurama'])],
    ['Dove',linear_regression_int(RevP['Number'],RevP['Dove']),linear_regression_slope(RevP['Number'],RevP['Dove'])],
    ['Fandor',linear_regression_int(RevP['Number'],RevP['Fandor']),linear_regression_slope(RevP['Number'],RevP['Fandor'])],
    ['Screambox',linear_regression_int(RevP['Number'],RevP['Screambox']),linear_regression_slope(RevP['Number'],RevP['Screambox'])]
    
], 
                     columns=['Subscription','intercept','Slope',])

In [ ]:
RevPrj

In [ ]:
Mon1=RevP.iloc[-1,6]+1
Year1=RevP.iloc[-1,6]+12
Year2=RevP.iloc[-1,6]+24
Year3=RevP.iloc[-1,6]+36
Year4=RevP.iloc[-1,6]+48


ProjTable=pd.DataFrame([
    [RevP['Number'].iloc[-2], RevP.iloc[-2]['Date'], RevP.iloc[-2]['CONtv'], RevP.iloc[-2]['Docurama'], RevP.iloc[-2]['Dove'], RevP.iloc[-2]['Fandor'], RevP.iloc[-2]['Screambox'] ],
    [RevP['Number'].iloc[-1], RevP.iloc[-1]['Date'], RevP.iloc[-1]['CONtv'], RevP.iloc[-1]['Docurama'], RevP.iloc[-1]['Dove'], RevP.iloc[-1]['Fandor'], RevP.iloc[-1]['Screambox']  ],
    # 1 Month later
    [Mon1 , '8-2022',   (Mon1*RevPrj.loc[RevPrj['Subscription']=='CONtv','Slope'].values[0]) + RevPrj.loc[RevPrj['Subscription']=='CONtv','intercept'].values[0],
                     (Mon1*RevPrj.loc[RevPrj['Subscription']=='Docurama','Slope'].values[0]) + RevPrj.loc[RevPrj['Subscription']=='Docurama','intercept'].values[0],
                     (Mon1*RevPrj.loc[RevPrj['Subscription']=='Dove','Slope'].values[0]) + RevPrj.loc[RevPrj['Subscription']=='Dove','intercept'].values[0],
                     (Mon1*RevPrj.loc[RevPrj['Subscription']=='Fandor','Slope'].values[0]) + RevPrj.loc[RevPrj['Subscription']=='Fandor','intercept'].values[0],
                     (Mon1*RevPrj.loc[RevPrj['Subscription']=='Screambox','Slope'].values[0]) + RevPrj.loc[RevPrj['Subscription']=='Screambox','intercept'].values[0]],
    # 1 Year Later
   [Year1 , '7-2023',   (Year1*RevPrj.loc[RevPrj['Subscription']=='CONtv','Slope'].values[0]) + RevPrj.loc[RevPrj['Subscription']=='CONtv','intercept'].values[0],
                     (Year1*RevPrj.loc[RevPrj['Subscription']=='Docurama','Slope'].values[0]) + RevPrj.loc[RevPrj['Subscription']=='Docurama','intercept'].values[0],
                     (Year1*RevPrj.loc[RevPrj['Subscription']=='Dove','Slope'].values[0]) + RevPrj.loc[RevPrj['Subscription']=='Dove','intercept'].values[0],
                     (Year1*RevPrj.loc[RevPrj['Subscription']=='Fandor','Slope'].values[0]) + RevPrj.loc[RevPrj['Subscription']=='Fandor','intercept'].values[0],
                     (Year1*RevPrj.loc[RevPrj['Subscription']=='Screambox','Slope'].values[0]) + RevPrj.loc[RevPrj['Subscription']=='Screambox','intercept'].values[0]],
    # 2 Years Later
    [Year2 , '7-2024',   (Year2*RevPrj.loc[RevPrj['Subscription']=='CONtv','Slope'].values[0]) + RevPrj.loc[RevPrj['Subscription']=='CONtv','intercept'].values[0],
                     (Year2*RevPrj.loc[RevPrj['Subscription']=='Docurama','Slope'].values[0]) + RevPrj.loc[RevPrj['Subscription']=='Docurama','intercept'].values[0],
                     (Year2*RevPrj.loc[RevPrj['Subscription']=='Dove','Slope'].values[0]) + RevPrj.loc[RevPrj['Subscription']=='Dove','intercept'].values[0],
                     (Year2*RevPrj.loc[RevPrj['Subscription']=='Fandor','Slope'].values[0]) + RevPrj.loc[RevPrj['Subscription']=='Fandor','intercept'].values[0],
                     (Year2*RevPrj.loc[RevPrj['Subscription']=='Screambox','Slope'].values[0]) + RevPrj.loc[RevPrj['Subscription']=='Screambox','intercept'].values[0]],
    #3 Years Later
    [Year3 , '7-2025',   (Year3*RevPrj.loc[RevPrj['Subscription']=='CONtv','Slope'].values[0]) + RevPrj.loc[RevPrj['Subscription']=='CONtv','intercept'].values[0],
                     (Year3*RevPrj.loc[RevPrj['Subscription']=='Docurama','Slope'].values[0]) + RevPrj.loc[RevPrj['Subscription']=='Docurama','intercept'].values[0],
                     (Year3*RevPrj.loc[RevPrj['Subscription']=='Dove','Slope'].values[0]) + RevPrj.loc[RevPrj['Subscription']=='Dove','intercept'].values[0],
                     (Year3*RevPrj.loc[RevPrj['Subscription']=='Fandor','Slope'].values[0]) + RevPrj.loc[RevPrj['Subscription']=='Fandor','intercept'].values[0],
                     (Year3*RevPrj.loc[RevPrj['Subscription']=='Screambox','Slope'].values[0]) + RevPrj.loc[RevPrj['Subscription']=='Screambox','intercept'].values[0]],
    #4 Years Later
    [Year4 , '7-2026',   (Year4*RevPrj.loc[RevPrj['Subscription']=='CONtv','Slope'].values[0]) + RevPrj.loc[RevPrj['Subscription']=='CONtv','intercept'].values[0],
                     (Year4*RevPrj.loc[RevPrj['Subscription']=='Docurama','Slope'].values[0]) + RevPrj.loc[RevPrj['Subscription']=='Docurama','intercept'].values[0],
                     (Year4*RevPrj.loc[RevPrj['Subscription']=='Dove','Slope'].values[0]) + RevPrj.loc[RevPrj['Subscription']=='Dove','intercept'].values[0],
                     (Year4*RevPrj.loc[RevPrj['Subscription']=='Fandor','Slope'].values[0]) + RevPrj.loc[RevPrj['Subscription']=='Fandor','intercept'].values[0],
                     (Year4*RevPrj.loc[RevPrj['Subscription']=='Screambox','Slope'].values[0]) + RevPrj.loc[RevPrj['Subscription']=='Screambox','intercept'].values[0]]
],
                       columns=['Number','Date','CONtv','Docurama','Dove','Fandor','Screambox'])

In [ ]:
(Mon1*RevPrj.iloc[0]['Slope']) + RevPrj.loc[RevPrj['Subscription']=='CONtv']['intercept'].values[0]

In [ ]:
ProjTable

# Subs Projections

In [ ]:
TotMSubs= TotMRevM.loc[:,:]

In [ ]:
TotMSubs.loc[TotMSubs['Subscription']=='CONtv','4-2020':]=TotMSubs.loc[TotMSubs['Subscription']=='CONtv','4-2020':].div(4.99)
TotMSubs.loc[TotMSubs['Subscription']=='Docurama','4-2020':]=TotMSubs.loc[TotMSubs['Subscription']=='Docurama','4-2020':].div(4.99)
TotMSubs.loc[TotMSubs['Subscription']=='Dove','4-2020':]=TotMSubs.loc[TotMSubs['Subscription']=='Dove','4-2020':].div(4.99)
TotMSubs.loc[TotMSubs['Subscription']=='Fandor','4-2020':]=TotMSubs.loc[TotMSubs['Subscription']=='Fandor','4-2020':].div(3.99)
TotMSubs.loc[TotMSubs['Subscription']=='Screambox','4-2020':]=TotMSubs.loc[TotMSubs['Subscription']=='Screambox','4-2020':].div(4.99)

In [ ]:
SubP= TotMSubs.set_index('Subscription').T
SubP.reset_index(inplace=True)
SubP.columns=['Date','CONtv','Docurama','Dove','Fandor','Screambox','Number']

In [ ]:
SubPrj= pd.DataFrame([
    ['CONtv',linear_regression_int(SubP['Number'],SubP['CONtv']),linear_regression_slope(SubP['Number'],SubP['CONtv'])],
    ['Docurama',linear_regression_int(SubP['Number'],SubP['Docurama']),linear_regression_slope(SubP['Number'],SubP['Docurama'])],
    ['Dove',linear_regression_int(SubP['Number'],SubP['Dove']),linear_regression_slope(SubP['Number'],SubP['Dove'])],
    ['Fandor',linear_regression_int(SubP['Number'],SubP['Fandor']),linear_regression_slope(SubP['Number'],SubP['Fandor'])],
    ['Screambox',linear_regression_int(SubP['Number'],SubP['Screambox']),linear_regression_slope(SubP['Number'],SubP['Screambox'])]
    
], 
                     columns=['Subscription','intercept','Slope',])

In [ ]:
SubP

In [ ]:
SubPrj

In [ ]:
TotMSubs

# Subs Projections (Last 6 Months)

In [ ]:
SubP6=SubP.iloc[-6:]

In [ ]:
Sub6Prj= pd.DataFrame([
    ['CONtv',linear_regression_int(SubP6['Number'],SubP6['CONtv']),linear_regression_slope(SubP6['Number'],SubP6['CONtv'])],
    ['Docurama',linear_regression_int(SubP6['Number'],SubP6['Docurama']),linear_regression_slope(SubP6['Number'],SubP6['Docurama'])],
    ['Dove',linear_regression_int(SubP6['Number'],SubP6['Dove']),linear_regression_slope(SubP6['Number'],SubP6['Dove'])],
    ['Fandor',linear_regression_int(SubP6['Number'],SubP6['Fandor']),linear_regression_slope(SubP6['Number'],SubP6['Fandor'])],
    ['Screambox',linear_regression_int(SubP6['Number'],SubP6['Screambox']),linear_regression_slope(SubP6['Number'],SubP6['Screambox'])]
    
], 
                     columns=['Subscription','intercept','Slope',])

In [ ]:
Sub6Prj

In [ ]:
SubP6

# Percent Rev increase by Year

In [ ]:
PercentSub= pd.DataFrame([
    #Year 1
    [ProjTable.iloc[3]['Number'], ProjTable.iloc[3]['Date']+'_+1Year_(LinReg)',   (ProjTable.iloc[3]['CONtv']-ProjTable.iloc[1]['CONtv'])/ProjTable.iloc[1]['CONtv'],
                                                                         (ProjTable.iloc[3]['Docurama']-ProjTable.iloc[1]['Docurama'])/ProjTable.iloc[1]['Docurama'],
                                                                         (ProjTable.iloc[3]['Dove']-ProjTable.iloc[1]['Dove'])/ProjTable.iloc[1]['Dove'],
                                                                         (ProjTable.iloc[3]['Fandor']-ProjTable.iloc[1]['Fandor'])/ProjTable.iloc[1]['Fandor'],
                                                                         (ProjTable.iloc[3]['Screambox']-ProjTable.iloc[1]['Screambox'])/ProjTable.iloc[1]['Screambox'] ],
    #Year 2 
    [ProjTable.iloc[4]['Number'], ProjTable.iloc[4]['Date']+'_+2Year_(LinReg)',   (ProjTable.iloc[4]['CONtv']-ProjTable.iloc[1]['CONtv'])/ProjTable.iloc[1]['CONtv'],
                                                                         (ProjTable.iloc[4]['Docurama']-ProjTable.iloc[1]['Docurama'])/ProjTable.iloc[1]['Docurama'],
                                                                         (ProjTable.iloc[4]['Dove']-ProjTable.iloc[1]['Dove'])/ProjTable.iloc[1]['Dove'],
                                                                         (ProjTable.iloc[4]['Fandor']-ProjTable.iloc[1]['Fandor'])/ProjTable.iloc[1]['Fandor'],
                                                                         (ProjTable.iloc[4]['Screambox']-ProjTable.iloc[1]['Screambox'])/ProjTable.iloc[1]['Screambox'] ],
    #Year 3
    [ProjTable.iloc[5]['Number'], ProjTable.iloc[5]['Date']+'_+3Year_(LinReg)',   (ProjTable.iloc[5]['CONtv']-ProjTable.iloc[1]['CONtv'])/ProjTable.iloc[1]['CONtv'],
                                                                         (ProjTable.iloc[5]['Docurama']-ProjTable.iloc[1]['Docurama'])/ProjTable.iloc[1]['Docurama'],
                                                                         (ProjTable.iloc[5]['Dove']-ProjTable.iloc[1]['Dove'])/ProjTable.iloc[1]['Dove'],
                                                                         (ProjTable.iloc[5]['Fandor']-ProjTable.iloc[1]['Fandor'])/ProjTable.iloc[1]['Fandor'],
                                                                         (ProjTable.iloc[5]['Screambox']-ProjTable.iloc[1]['Screambox'])/ProjTable.iloc[1]['Screambox'] ],
    #Year 4
    [ProjTable.iloc[6]['Number'], ProjTable.iloc[6]['Date']+'_+4Year_(LinReg)',   (ProjTable.iloc[6]['CONtv']-ProjTable.iloc[1]['CONtv'])/ProjTable.iloc[1]['CONtv'],
                                                                         (ProjTable.iloc[6]['Docurama']-ProjTable.iloc[1]['Docurama'])/ProjTable.iloc[1]['Docurama'],
                                                                         (ProjTable.iloc[6]['Dove']-ProjTable.iloc[1]['Dove'])/ProjTable.iloc[1]['Dove'],
                                                                         (ProjTable.iloc[6]['Fandor']-ProjTable.iloc[1]['Fandor'])/ProjTable.iloc[1]['Fandor'],
                                                                         (ProjTable.iloc[6]['Screambox']-ProjTable.iloc[1]['Screambox'])/ProjTable.iloc[1]['Screambox'] ],
    
], 
    columns=['Number','Date','CONtv','Docurama','Dove','Fandor','Screambox'])

In [ ]:
PercentSub= PercentSub.loc[:,('Date','CONtv','Docurama','Dove','Fandor','Screambox')]
PercentSub= PercentSub.set_index('Date').T


In [ ]:
PercentSub

In [ ]:
PercentSub.reset_index(inplace=True)

In [ ]:
PercentSub.columns=['Subscription','+1Year%_(LinReg)','+2Year%_(LinReg)','+3Year%_(LinReg)','+4Year%_(LinReg)']

In [ ]:
PercentSub

In [ ]:
Rev1['Yearly Avg']= Rev1['All Platforms monthly Rev Avg (10/8 estimate)']*12

In [ ]:
Rev1=pd.merge(Rev1,PercentSub, on='Subscription', how='left')

In [ ]:
Rev1['+1 Year Proj.']= Rev1['Yearly Avg']* (1+ Rev1['+1Year%_(LinReg)'])
Rev1['+2 Year Proj.']= Rev1['Yearly Avg']* (1+ Rev1['+2Year%_(LinReg)'])
Rev1['+3 Year Proj.']= Rev1['Yearly Avg']* (1+ Rev1['+3Year%_(LinReg)'])
Rev1['+4 Year Proj.']= Rev1['Yearly Avg']* (1+ Rev1['+4Year%_(LinReg)'])

#New Rev
Rev1['+1 Year Proj. (New Rev)']= Rev1['+1 Year Proj.'] - Rev1['Yearly Avg']
Rev1['+2 Year Proj. (New Rev)']= Rev1['+2 Year Proj.'] - Rev1['+1 Year Proj.']
Rev1['+3 Year Proj. (New Rev)']= Rev1['+3 Year Proj.'] - Rev1['+2 Year Proj.']
Rev1['+4 Year Proj. (New Rev)']= Rev1['+4 Year Proj.'] - Rev1['+3 Year Proj.']

In [ ]:
Rev1

In [ ]:
Content2= Content2.loc[:,('Subscription','Year','Date','Month','Month Abv','ID','TV/MOVIE','Title',
                          'Amzn Min Streamed','Amzn Streams','Amzn Min/Stream','Roku Min Streamed','Roku Streams','Roku Min/Stream',
                          'Total Min Streamed','Percentile Rank (Min)','Total Streams','Min/Stream','Content Month Rev')]

# Plots

In [ ]:
ContDist =Content2.loc[:,('Subscription','Date','Total Min Streamed')]

In [ ]:
CDSx= ContDist.loc[(ContDist['Subscription']=='Screambox') & (ContDist['Date']== 'May-2022'),:]

In [ ]:
CDS= CDSx.loc[:,:].reset_index()

In [ ]:
CDS['Percent']= (CDS['Total Min Streamed']/CDS['Total Min Streamed'].sum())
CDS.sort_values(by='Total Min Streamed', inplace=True, ascending=False)

In [ ]:
CDS['PErcentile']= CDS.groupby(by=['Subscription', 'Date'])['Total Min Streamed'].rank(pct=True)

In [ ]:
CDS['rank']= CDS.groupby(by=['Subscription', 'Date'])['Total Min Streamed'].rank(ascending=False, method='first')

In [ ]:
CDS['Percent of Total Count (Lrgst to Smlst)']= (CDS['rank']/705)*100

In [ ]:
CDS['Cumulative Sum (Minutes)']=CDS['Total Min Streamed'].cumsum()

In [ ]:
CDS['Percent of Total Min (Lrgst to Smlst)']= CDS['Cumulative Sum (Minutes)']/ CDS['Cumulative Sum (Minutes)'].max()

In [ ]:
CDS

In [ ]:
fig, ax = plt.subplots(figsize=(19,10))
x1= CDS['Percent of Total Count (Lrgst to Smlst)']
y1= CDS['Percent of Total Min (Lrgst to Smlst)']

ax.plot(x1,y1)
plt.show()

In [ ]:
x= CDS['Percent']
fig, ax= plt.subplots(figsize=(19,10))
ax.hist(x, bins=50)
plt.show()

# Last Look

In [ ]:
Subs.head(10)

In [ ]:
Content2

In [ ]:
CM

In [ ]:
SubP

In [ ]:
SubPrj

In [ ]:
SubP6

In [ ]:
Sub6Prj

In [ ]:
Rev1

In [ ]:
output= pd.ExcelWriter('Amazon_Roku SVOD '+Monthsinc+' out.xlsx', engine='xlsxwriter')

Subs.to_excel(output, sheet_name= 'Subscriptions',index=False)
Content2.to_excel(output, sheet_name= 'Content',index=False)
CM.to_excel(output, sheet_name='Monthly Title Trend', index=False)
SubP.to_excel(output, sheet_name='Sub Projections', index=False)
SubPrj.to_excel(output, sheet_name='Sub Slp_int', index=False)
SubP6.to_excel(output, sheet_name='Sub6 Projections', index=False)
Sub6Prj.to_excel(output, sheet_name='Sub6 Slp_int', index=False)
Rev1.to_excel(output, sheet_name='Revenue', index=False)

output.save()